<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/google_colab/generation_generative_model_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install datasets
%pip install sentence_transformers
%pip install transformers
%pip install torch
%pip install tqdm
%pip install scikit-learn
%pip install accelerate
%pip install peft
%pip install bitsandbytes
%pip install trl
%pip install sentencepiece
%pip install triton

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
template_tokenizer = AutoTokenizer.from_pretrained(model_id)


def format_prompt(example):
    system = "<|system|>\n" + example["system"] + "</s>\n"
    prompt = "<|user|>\n" + example["input"] + "</s>\n<|assistant|>\n"
    chosen = example["chosen"] + "</s>\n"
    rejected = example["rejected"] + "</s>\n"

    return {"prompt": system + prompt, "chosen": chosen, "rejected": rejected}


dpo_dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train")
dpo_dataset = dpo_dataset.filter(
    lambda x: x["status"] != "tie"
    and x["chosen_score"] >= 8
    and not x["in_gsm8k_train"]
)
dpo_dataset = dpo_dataset.map(format_prompt, remove_columns=dpo_dataset.column_names)

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlama-1.1B-qlora"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

model = AutoPeftModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
    device_map="auto",
)

merged_model = model.merge_and_unload()

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "k_proj",
        "gate_proj",
        "v_proj",
        "up_proj",
        "q_proj",
        "o_proj",
        "down_proj",
    ],
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from trl import DPOConfig

output_dir = "./results"

training_arguments = DPOConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio=0.1,
)

In [ ]:
from trl import DPOTrainer

dpo_trainer = DPOTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=512,
)

dpo_trainer.train()
dpo_trainer.model.save_pretrained("TinyLlama-1.1B-dpo-qlora")

In [ ]:
from peft import PeftModel

peft_model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora", low_cpu_mem_usage=True, device_map="auto"
)
sft_model = peft_model.merge_and_unload()
dpo_model = PeftModel.from_pretrained(
    sft_model, "TinyLlama-1.1B-dpo-qlora", device_map="auto"
)
dpo_model = dpo_model.merge_and_unload()